### SIF

#### 准备
##### 词向量文件

- [词向量数据集(中文)](https://github.com/Embedding/Chinese-Word-Vectors)

In [43]:
wv_zn = open('data/sgns.merge.word', 'r', encoding='utf-8')
total_words_num, dimention = map(lambda num : int(num), wv_zn.readline().split())

print(f'总词数：{total_words_num}；词向量维度：{dimention}')

for i in range(1):
    print(wv_zn.readline())

总词数：1292607；词向量维度：300
， 0.102387 0.111146 0.263319 0.023152 -0.016981 0.024967 -0.019429 0.148500 0.044953 0.231089 -0.195045 -0.060415 0.229814 0.125847 0.174458 0.186934 0.047486 0.122592 -0.139524 0.219931 0.078783 0.144782 -0.160593 -0.081857 0.083243 0.058926 0.177047 -0.171374 0.018400 -0.027682 -0.004983 -0.076983 0.131640 -0.113488 0.167113 -0.121429 0.063881 0.171087 -0.145927 0.083276 -0.170163 -0.074225 0.103492 0.093889 -0.155911 0.163988 -0.182002 0.157361 -0.191924 0.089651 0.068731 0.142282 -0.113051 0.132194 -0.326704 0.131645 -0.046067 -0.199591 -0.263973 -0.104769 -0.048914 0.014195 -0.019854 0.056857 0.059569 0.091732 0.083272 -0.167209 -0.033752 0.211743 0.035805 -0.000383 -0.089494 -0.012996 0.137302 -0.261788 0.087270 -0.167092 -0.279155 0.047554 -0.159121 -0.149205 0.235909 -0.279607 0.054076 -0.081279 0.121545 0.167018 -0.118002 0.371543 -0.148008 0.089215 0.042067 0.046976 -0.012934 0.140683 0.182944 -0.073797 0.193460 -0.133254 0.026854 -0.080276 0.144142 -0.3

##### 词频
中文版的词向量并没有提供词频，作者给出的建议是使用Zipf's Law来估计词频，参考[issue#72](https://github.com/Embedding/Chinese-Word-Vectors/issues/72)，为了和SIF的接口对接，我们使用Zipf's Law来生成估计的词频文件
Zipf's Law指出：
> Zipf's law states that given a large sample of words used, the frequency of any word is inversely proportional to its rank in the frequency table.

也就是说某个词与出现的频率$f_i$与它在词频表（词语出现频率从高到低排序的序列）中的序号$rank_i$成反比：
$$
f_i = \frac{k}{rank_i}
$$

这其中$k$是一个常数，我们假设词表中最后一个出现的词语的出现频数为1，那么$k = 1 \times total\_words\_number$

In [44]:
wv_zn_freq = open('data/sgns.merge.word.freq', 'w+', encoding='utf-8')
wv_zn.seek(0, 0)
wv_zn.readline()
k = total_words_num
for rank in range(1, total_words_num + 1):
    word = wv_zn.readline().split()[0]
    freq = k // rank
    wv_zn_freq.write(f'{word} {freq}\n')
    if rank % 100000 == 0:
        print(f'{rank}/{total_words_num}')
wv_zn_freq.close()

100000/1292607
200000/1292607
300000/1292607
400000/1292607
500000/1292607
600000/1292607
700000/1292607
800000/1292607
900000/1292607
1000000/1292607
1100000/1292607
1200000/1292607


#### 运行SIF示例

In [45]:
# ! pip install -r SIF/requirements.txt

In [46]:
from SIF.src import data_io, params, SIF_embedding

# input
wordfile = 'data/sgns.merge.word' # word vector file, can be downloaded from GloVe website
weightfile = 'data/sgns.merge.word.freq' # each line is a word and its frequency
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 0 # number of principal components to remove in SIF weighting scheme


In [47]:
# load word vectors
(words, We) = data_io.getWordmap(wordfile, limit=1000)

invalid line 0, skipped, start with 1292607
0


In [48]:
# load word weights
word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word

In [49]:
sentences = [
    "网络抓取的微信公众号的文章，已经去除HTML，只包含了纯文本。每行一篇，是JSON格式，name是微信公众号名字，account是微信公众号ID，title是题目，content是正文。数据用zip分卷压缩过的, 没有密码。预览可以看preview.json。目前数据大约3G，数据会定期更新增加。网络抓取的微信公众号的文章。", 	
    "27日上午9点20分左右，消防救援人员在技术组指定的重点区域深度搜寻，人工挖掘出一橙色圆柱状物体，现场勘查的民航事故调查人员发现后立即取出，经确认为飞行数据记录器存储单元。发现位置在距主要撞击点以东约40米的坡面，地表下深约1.5米处。经检查，记录器其他部分损毁严重，数据存储单元外观较为完好。目前，该记录器已送往专业实验室进行译码工作。（人民日报）"]

In [50]:
# load sentences
import jieba
spliter = lambda sentence : list(jieba.cut(sentence))


In [51]:
x, m = data_io.sentences2idx(sentences, words, spliter=spliter) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
print(x)

w = data_io.seq2weight(x, m, weight4ind) # get word weights

[[  525 33227     1  2035  1709   170     1  1415     0   115 11121 15534
      0   206  2497     7  1707  9151     2 90612 99971     0     8 99971
   4854     0 42582     8  2035  1709   170  2271     0 99971     8  2035
   1709   170 15985     0 45329     8  7251     0 93427     8  7568     2
    279    92 94628 99971   163     1    18 99971    82  1506     2 38139
     75   199 99971    44 99971     2   129   279  2472 99971     0   279
     70  2756  2856   283     2   525 33227     1  2035  1709   170     1
   1415     2     0     0     0     0     0     0     0     0     0     0
      0     0]
 [ 1491    34  1896   197   161   289   296   603     0  5968  3618   398
      4   147   789  2835     1   498   835  1941 13440     0  3255 45860
     15 16427 64710  6511     0   705 13589     1  9189  2466   821   398
    266    42  1342  6320     0   404  2205    12 99971 99971 99971     2
    266  1028     4  1732   121 12250   161  9788   238  1001   272     1
  98383     0 12649    

In [52]:
# set parameters
from SIF.src import params
params = params.params()
params.rmpc = rmpc

In [53]:
# get SIF embedding
embedding = SIF_embedding.SIF_embedding(We, x, w, params) # embedding[i,:] is the embedding for sentence i
print(embedding.shape)

(2, 300)


In [54]:
import numpy as np

def get_cos_similar(v1: list, v2: list, normalize=False):
    num = np.dot(v1, v2)  # 向量点乘
    # print(num)
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)  # 求模长的乘积
    # print(denom)
    cos_val = (num / denom) if denom != 0 else 0
    if(normalize):
        return (cos_val + 1) / 2
    else:
        return cos_val

### 文档相似度类设计

In [55]:
from SIF.src import params
class DocSimilarity:
    def __init__(self, wordfile, weightfile, spliter, weightpara=1e-3, rmpc=0, limit_words=None, limit_weights=None) -> None:
        '''
        Args:
            wordfile (str): word vector file
            weightfile (str): each line is a word and its frequency
            weightpara (float): the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
            rmpc (int): number of principal components to remove in SIF weighting scheme
            spliter (func): 用于分词的方法
            limit_words (int): 用于限制加载词向量的数量（词向量非常消耗内存）
            limit_weights ():用于限制加载词频的数量
        '''
        # load word vectors
        (words, We) = data_io.getWordmap(wordfile, limit=limit_words)
        # load word weights
        word2weight = data_io.getWordWeight(weightfile, weightpara, limit=limit_weights) # word2weight['str'] is the weight for the word 'str'
        weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word
        # set parameters
        para = params.params()
        para.rmpc = rmpc

        self.spliter = spliter ## 如果是中文，需要传入分词方法
        self.words = words
        self.We = We
        self.word2weight = word2weight
        self.weight4ind = weight4ind
        self.params = para
        self.pc = None
        self.npc = rmpc



    def refresh_pc_with_sentences(self, sentences, npc=1):
        '''
        Args:
        	sentences (list(str)): 用来计算主成分的句子组
            npc (int): 主成分的数量（计算句子向量的时候会去除在这些主成分方向上的投影）
        Return:
        	主成分
        '''
        self.npc = npc
        #### 将句子组变成一个矩阵，矩阵的每一行都是一个句子向量，它们还没有被去除主成分上的投影
        x, m = data_io.sentences2idx(sentences, self.words, spliter=self.spliter) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
        w = data_io.seq2weight(x, m, self.weight4ind) # get word weights
        self.pc = SIF_embedding.compute_sentences_pc(self.We, x, w, self.npc)
        return self.pc

    def get_embedding_with_exist_pc(self, sentences):
        '''
        计算给定句子组中每个句子的向量，会将句子向量在之前计算的主成分方向上的投影去除
        Args:
        	sentences (list(str)): 句子组
        Return:
        	句子矩阵
        '''
        if self.pc is None:
            return None
        x, m = data_io.sentences2idx(sentences, self.words, spliter=self.spliter) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
        w = data_io.seq2weight(x, m, self.weight4ind) # get word weights
        embedding = SIF_embedding.SIF_embedding_with_exists_pc(self.We, x, w, self.pc, self.npc)
        return embedding

    def cos_similarity_vector_with_exist_pc(self, sentences, normalize=False):
        """
        Args:
            sentences (list(str)): 一组句子
            normalize (boolean): 相似度是否归一化
        Return:
            返回第一个句子和句子数组中每一个句子的相似度，使用已有的pc
        """
        if self.pc is None:
            return None
        sentence_num = len(sentences)
        embedding = self.get_embedding_with_exist_pc(sentences)
        vector = np.zeros((sentence_num))
        for i in range(sentence_num):
            vector[i] = get_cos_similar(embedding[0], embedding[i], normalize=normalize)
        return vector


#### 使用示例

In [56]:
doc_similarity = DocSimilarity(wordfile, weightfile, spliter, limit_words=100000)

invalid line 0, skipped, start with 1292607
0


In [57]:
sentences_to_calculate_pc = [
    "3个数据集合其中dev或者train数据量和batch_size能整除的时候就回出现问题，DatasetIterater函数这块一个bug，会导致爆出问题",
    "DatasetIterater函数这块有个问题，batch_size能整除dev或者train数据量的话就会报错",
    "“Elevator 电梯楼层”组件，有这样一种情况，在城市列表中，有热门城市一项；一个城市的id固定，但是它又是热门城市，造成点击后会出现两个高亮色。",
    "使用双色BarCharts，但是柱状图的位置有3成左右的几率是乱掉了，这个时候如果拖动鼠标调整一下浏览器窗口大小，柱状图就会排列正常了",
    "在“第一个数”中输入10 在第二个数中输入0 点击除法按钮 在错误提示框中点击确定按钮 预期结果为：“错误提示框”关闭，程序继续运行 实际结果：程序关闭",
    "新增和编辑行时无法成功， 报错了。给操作的小图标一个title提示，以便鼠标浮动上去时能够显示该图标是干嘛的",
    "假设文件夹里含有文件，比如pdf文件，同时这个pdf文件未被导入到索引中。",
    "当这个文件夹被剪切走时，会出现这样的现象：这个文件夹并不能被剪切走，文件夹依然存在，只是未被加入到索引中，而被粘贴出现的新文件夹一切正常。",
    "如果那个pdf文件已经被导入了索引中，剪切粘贴功能似乎就一切正常。解决办法：在剪切前，把文件夹底下的内容自动导入到索引中？我进一步测试了，如果文件夹里含有未被导入索引的文件，删除文件夹功能也失效。",
    "但是柱状图的位置有一部分是乱掉的，这个时候只有用拖动鼠标的方式调整一下浏览器窗口大小，柱状图才会排列正常了",
    "例如系统已存在可正常登录的用户名称为Chang，如果使用chang作为用户名登录，系统提示登录成功，但是仍然会返回到登录界面，实际上是没有登录成功的。",
    "主界面中连续快速点击“更改设置”按钮，程序会卡死首先，进入应用主界面；其次，连续点击右上角的“更改设置”按钮，注意一定要点的飞快。十几次后程序会无法响应，强制退出。",
    "主界面中快速点“更改设置”按钮就会卡死首先，进入应用主界面然后连续点击右上角的“更改设置”按钮，要点的飞快。十几次后会无法响应",
    "内置跳转小程序打不开在“发现页面”点击左下角“开心一下”，页面发生跳转，不过一直是白屏，无法打开",
    "去掉保存和取消的文字， 取消图标用减号-显示；对于新增未保存（区别于默认传进来的数据和已编辑过的数据）的数据点击取消时应该将该列删除掉",
    "[集成管理]模块的[项目注册管理]页面， 查看项目立项信息时，出现“未能加载类型“ProjectGeneralView”错误",
    "在[项目管理]—> [项目基本信息管理]页面下；2）选择[已审核]TAB页； 3）在GRID列表中，选择任意一条项目立项信息，点击[查看]按钮；4）系统页面弹出：按钮报错信息“"
]

sentences_to_test_algrithm_1 = [
    "无法发布测试;进入首页，以发包方身份登陆，点击右上角菜单中的发布测试，填写测试信息，点击发布，预期系统提示发布成功，但实际上系统没有任何提示，测试页并没有发布", #句子1
    "发布测试出错；发包方登录后从导航栏菜单进入发布测试页面后，输入测试描述，所需人数，起止时间，并上传测试文档以后点击发布，系统无响应", #句子2
    "无法提交报告，以众包工人身份登录，在主页中选择正在进行一栏，选择需要提交报告的测试，填写报告信息，点击提交报告，预期系统提示提交成功，但实际上系统没有任何提示，报告并未提交" #句子3
]

sentences_to_test_algrithm_2 = [
    "无法提交报告，以众包工人身份登录，在主页中选择正在进行一栏，选择需要提交报告的测试，填写报告信息，点击提交报告，预期系统提示提交成功，但实际上系统没有任何提示，报告并未提交", #句子3
    "无法发布测试;进入首页，以发包方身份登陆，点击右上角菜单中的发布测试，填写测试信息，点击发布，预期系统提示发布成功，但实际上系统没有任何提示，测试页并没有发布", #句子1
    "发布测试出错；发包方登录后从导航栏菜单进入发布测试页面后，输入测试描述，所需人数，起止时间，并上传测试文档以后点击发布，系统无响应", #句子2
]

In [58]:
doc_similarity.refresh_pc_with_sentences(sentences_to_calculate_pc)
similaritys_1 = doc_similarity.cos_similarity_vector_with_exist_pc(sentences_to_test_algrithm_1, normalize=True)
similaritys_2 = doc_similarity.cos_similarity_vector_with_exist_pc(sentences_to_test_algrithm_2, normalize=True)
print(f'                句子2                   句子3')
print(f'句子1   {similaritys_1[1]}      {similaritys_1[2]}')
print(f'句子2                           {similaritys_2[2]}')
print(similaritys_1)
print(similaritys_2)

                句子2                   句子3
句子1   0.9140273837372119      0.8601595233279353
句子2                           0.8349675055440855
[1.         0.91402738 0.86015952]
[1.         0.86015952 0.83496751]
